# Hyperparameter Tuning using HyperDrive

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os
import shutil
import joblib

## Dataset

The dataset used for this project is the [Heart Failure Prediction dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) taken from Kaggle.

This dataset contains 12 features that can be used to predict mortality by heart failure:
- age: Age of the patient 
- amaemia: Decrease of red blood cells or hemoglobin 
- creatinine_phosphokinase: Level of the CPK enzyme in the blood (mcg/L)
- diabetes: If the patient has diabetes
- ejection_fraction: Percentage of blood leaving the heart at each contraction 
- high_blood_pressure: If the patient has hypertension
- platelets: Platelets in the blood (kiloplatelets/mL)
- serum_creatinine: Level of serum creatinine in the blood (mg/dL)
- serum_sodium: Level of serum sodium in the blood (mEq/L)
- sex: Woman or man
- smoking: If the patient smokes or not
- time: Follow-up period (days)

The target column is DEATH_EVENT which tells if the patient deceased during the follow-up period

In [2]:
ws = Workspace.from_config()
experiment_name = 'heart-failure-hyperdrive'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-177051
Azure region: southcentralus
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-177051


In [3]:
#Create compute cluster
compute_cluster_name= "my-compute"

#Check if compute cluster already exists
try:
    compute_cluster=ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing cluster, use it...")
except ComputeTargetException:
    print("Creating new cluster...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=5)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    
compute_cluster.wait_for_completion(show_output=True)

Creating new cluster...
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

The Scikit-learn model is a Logistic Regression Model which is trained using a custom coded script- train.py. The dataset is fetched from a url as a TabularDataset. The hyperparameters chosen for the Scikit-learn model are regularization strength (C) and max iterations (max_iter). The trained model is scored against 20% data selected from the original dataset.

The hyperparameter tuning using HyperDrive requires several steps- Defining parameter search space, defining a sampling method, choosing a primary metric to optimize and selecting an early stopping policy.

The parameter sampling method used for this project is Random Sampling. It randomly selects the best hyperparameters for the model, that way the entire search space does not need to be searched. The random sampling method saves on time and is a lot faster than grid sampling and bayesian sampling which are recommended only if you have budget to explore the entire search space

The early stopping policy used in this project is Bandit Policy which is based on a slack factor (0.1 in this case) and an evaluation interval (1 in this case). This policy terminates runs where the primary metric is not within the specified slack factor as compared to the best performing run. This would save on time and resources as runs which won't potentially lead to good results would be terminated early.

In [4]:
# Create an early termination policy.
early_termination_policy = BanditPolicy(
    evaluation_interval=1,
    slack_factor= 0.1
)

# Create the different params that will be needed during training
param_sampling = RandomParameterSampling(
    {
        "--C": uniform(0.001, 100),
        "--max_iter": choice(50, 75, 100, 125, 150)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")
    
script_folder = './training'
os.makedirs(script_folder, exist_ok=True)

shutil.copy('./train.py', script_folder)

# Create estimator and hyperdrive config
estimator = SKLearn(
    source_directory= script_folder,
    compute_target= compute_cluster,
    entry_script= "train.py",
    vm_size="Standard_D2_V2",
    vm_priority="lowpriority"
)

hyperdrive_run_config = HyperDriveConfig(
    estimator=estimator,
    hyperparameter_sampling= param_sampling,
    policy= early_termination_policy,
    primary_metric_name= "Accuracy",
    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=5
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [5]:
# Submit the experiment
hyperdrive_run=experiment.submit(config=hyperdrive_run_config)

## Run Details

In [6]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
hyperdrive_run.wait_for_completion(show_output= True)

RunId: HD_7bb2c407-724e-497f-98af-188126a78904
Web View: https://ml.azure.com/runs/HD_7bb2c407-724e-497f-98af-188126a78904?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-177051/workspaces/quick-starts-ws-177051&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-01-20T15:34:48.953305][API][INFO]Experiment created<END>\n""<START>[2022-01-20T15:34:49.838502][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space<END>\n""<START>[2022-01-20T15:34:50.496694][GENERATOR][INFO]Successfully sampled '5' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_7bb2c407-724e-497f-98af-188126a78904
Web View: https://ml.azure.com/runs/HD_7bb2c407-724e-497f-98af-188126a78904?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-177051/workspaces/quick-starts-ws-177051&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_7bb2c407-724e-497f-98af-188126a78904',
 'target': 'my-compute',
 'status': 'Completed',
 'startTimeUtc': '2022-01-20T15:34:48.66692Z',
 'endTimeUtc': '2022-01-20T15:46:30.650932Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'e025a677-ddc2-456d-87d8-60c2116362a3',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1063-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.36.0',
  'space_size': 'infinite_space_size',
  'score': '0.8833333333333333',
  'best_child_run_id': 'HD_7bb2c407-724e-497f-98af-188126a78904_9',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg177051.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_7bb2c407-724e-497f-98af-188126a

## Best Model

In [8]:
# Get best model and diplay all details
best_run= hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics=best_run.get_metrics()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())
print('Best Run Accuracy:',best_run_metrics['Accuracy'])

['--C', '59.302696995505244', '--max_iter', '75']
['azureml-logs/55_azureml-execution-tvmps_96c6a5e871f669a8762920a139eb2a5f0b95cf167dff70b4fb290d7c3144a512_d.txt', 'azureml-logs/65_job_prep-tvmps_96c6a5e871f669a8762920a139eb2a5f0b95cf167dff70b4fb290d7c3144a512_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_96c6a5e871f669a8762920a139eb2a5f0b95cf167dff70b4fb290d7c3144a512_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/111_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']
Best Run Accuracy: 0.8833333333333333


In [9]:
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_96c6a5e871f669a8762920a139eb2a5f0b95cf167dff70b4fb290d7c3144a512_d.txt',
 'azureml-logs/65_job_prep-tvmps_96c6a5e871f669a8762920a139eb2a5f0b95cf167dff70b4fb290d7c3144a512_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_96c6a5e871f669a8762920a139eb2a5f0b95cf167dff70b4fb290d7c3144a512_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/111_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/model.joblib']

In [10]:
# Save the best model
model=best_run.register_model(model_name='heart-failure-sklearn', model_path='outputs/model.joblib')
best_run.download_file('/outputs/model.joblib', 'hyperdrive_model.joblib')

In [11]:
# Clean up allocated resources
compute_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

